In [2]:
from itertools import combinations
from z3 import *
import math

In [4]:
def overlapping(s, size_i, size_j, limit, relation, p, i, j, perm, perm_i, perm_j):
    if size_i >= limit - size_j + 1: 
        s.add(Or(perm_i(), perm_j(), Not(relation[i][j])))
    else:
        s.add(Or(perm_i(), perm_j(), Not(relation[i][j]), Not(p[j][size_i-1])))       
        for elx in range(limit - size_i + 1):
            if elx + size_i + 1 >= limit - size_j + 1:
                s.add(Or(perm_i(), perm_j(), Not(relation[i][j]), p[i][elx]))
                break
            else:
                s.add(Or(perm_i(), perm_j(), Not(relation[i][j]), p[i][elx], Not(p[j][elx + size_i])))

def two_d_spp(r_sizes, plate_width, lower_bound, upper_bound, n_of_rectangles):

    height = int((upper_bound + lower_bound)/2)
    print(height)
    lr = [[Bool(f"lr_{i}_{j}") for j in range(1, n_of_rectangles+1)] for i in range(1, n_of_rectangles+1)]
    ud = [[Bool(f"ud_{i}_{j}") for j in range(1, n_of_rectangles+1)] for i in range(1, n_of_rectangles+1)]
    px = [[Bool(f"px_{i+1}_{e}") for e in range(plate_width - (min(r_sizes[i][0],r_sizes[i][1])) + 1 )]
          for i in range(n_of_rectangles)]    
    py = [[Bool(f"py_{i+1}_{e}") for e in range(height - (min(r_sizes[i][0],r_sizes[i][1])) + 1 )]
          for i in range(n_of_rectangles)] 
    
    perm = [Bool(f"perm_{i}") for i in range(1, n_of_rectangles+1)]
    
    s = Solver()
    
    for i in range(n_of_rectangles):
        for elx in range(len(px[i])-1):
            s.add(Or(Not(px[i][elx]), px[i][elx+1]))
        for ely in range(len(py[i])-1): 
            s.add(Or(Not(py[i][ely]), py[i][ely+1]))
    for i in range(n_of_rectangles):
        for j in range(i+1, n_of_rectangles):
            s.add(Or(lr[i][j], lr[j][i], ud[i][j], ud[j][i]))

            
    for i in range(n_of_rectangles):
        for j in range(i+1, n_of_rectangles):
            w_i= r_sizes[i][0]
            w_j= r_sizes[j][0]
            h_i= r_sizes[i][1]
            h_j= r_sizes[j][1]
            
            overlapping(s, w_i, w_j, plate_width, lr, px, i, j, perm, lambda: perm[i], lambda: perm[j])
            overlapping(s, h_i, w_j, plate_width, lr, px, i, j, perm, lambda: Not(perm[i]), lambda: perm[j])
            overlapping(s, w_i, h_j, plate_width, lr, px, i, j, perm, lambda: perm[i], lambda: Not(perm[j]))
            overlapping(s, h_i, h_j, plate_width, lr, px, i, j, perm, lambda: Not(perm[i]), lambda: Not(perm[j]))
            
            overlapping(s, w_j, w_i, plate_width, lr, px, j, i, perm, lambda: perm[i], lambda: perm[j])
            overlapping(s, h_j, w_i, plate_width, lr, px, j, i, perm, lambda: perm[i], lambda: Not(perm[j]))
            overlapping(s, w_j, h_i, plate_width, lr, px, j, i, perm, lambda: Not(perm[i]), lambda: perm[j])
            overlapping(s, h_j, h_i, plate_width, lr, px, j, i, perm, lambda: Not(perm[i]), lambda: Not(perm[j]))
            
            overlapping(s, h_i, h_j, height, ud, py, i, j, perm, lambda: perm[i], lambda: perm[j])
            overlapping(s, w_i, h_j, height, ud, py, i, j, perm, lambda: Not(perm[i]), lambda: perm[j])
            overlapping(s, h_i, w_j, height, ud, py, i, j, perm, lambda: perm[i], lambda: Not(perm[j]))
            overlapping(s, w_i, w_j, height, ud, py, i, j, perm, lambda: Not(perm[i]), lambda: Not(perm[j]))
            
            overlapping(s, h_j, h_i, height, ud, py, j, i, perm, lambda: perm[i], lambda: perm[j])
            overlapping(s, w_j, h_i, height, ud, py, j, i, perm, lambda: perm[i], lambda: Not(perm[j]))
            overlapping(s, h_j, w_i, height, ud, py, j, i, perm, lambda: Not(perm[i]), lambda: perm[j])
            overlapping(s, w_j, w_i, height, ud, py, j, i, perm, lambda: Not(perm[i]), lambda: Not(perm[j]))     
            
            
            
            
    s.add(And(Not(lr[1][0]), Not(ud[1][0])))       
    
    if s.check() == sat and height == lower_bound:
        m = s.model()
        return (height,  [item for sublist in px for item in sublist if m.evaluate(item)==True], 
                         [item for sublist in py for item in sublist if m.evaluate(item)==True],
                         [item for sublist in lr for item in sublist if m.evaluate(item)==True],
                         [item for sublist in ud for item in sublist if m.evaluate(item)==True],
                         [item for item in perm if m.evaluate(item)==True]
               )
    
                
       
    if s.check()==sat:
        return two_d_spp(r_sizes, plate_width, lower_bound, height, n_of_rectangles)
    return two_d_spp(r_sizes, plate_width, height+1, upper_bound, n_of_rectangles)    
W = 12
n_of_rectangles = 8
sizes = [[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[3,9],[6,3]]
            
#W = 9;
#n_of_rectangles = 5
#sizes = [[3,3],[3,4],[3,5],[3,6],[3,9]]
#W = 20
#n_of_rectangles = 14
#sizes = [[3,3], [3,4], [3,5], [3,6], [3,7], [3,8], [3,9],[3,10],[3,11],[3,17],[4,3],[4,9],[4,11],[4,17]]
lower_bound = max([sizes[i][1] for i in range(n_of_rectangles)])
upper_bound = sum([sizes[i][1] for i in range(n_of_rectangles)])
two_d_spp(sizes, W, lower_bound, upper_bound, n_of_rectangles)

68
[]
42
[]
29
[]
23
[]
20
[]
18
[]
19
[]
20
[]


(20,
 [px_1_4,
  px_1_5,
  px_1_6,
  px_1_7,
  px_1_8,
  px_1_9,
  px_1_10,
  px_1_11,
  px_1_12,
  px_1_13,
  px_1_14,
  px_1_15,
  px_1_16,
  px_1_17,
  px_2_4,
  px_2_5,
  px_2_6,
  px_2_7,
  px_2_8,
  px_2_9,
  px_2_10,
  px_2_11,
  px_2_12,
  px_2_13,
  px_2_14,
  px_2_15,
  px_2_16,
  px_2_17,
  px_3_4,
  px_3_5,
  px_3_6,
  px_3_7,
  px_3_8,
  px_3_9,
  px_3_10,
  px_3_11,
  px_3_12,
  px_3_13,
  px_3_14,
  px_3_15,
  px_3_16,
  px_3_17,
  px_4_10,
  px_4_11,
  px_4_12,
  px_4_13,
  px_4_14,
  px_4_15,
  px_4_16,
  px_4_17,
  px_5_13,
  px_5_14,
  px_5_15,
  px_5_16,
  px_5_17,
  px_6_8,
  px_6_9,
  px_6_10,
  px_6_11,
  px_6_12,
  px_6_13,
  px_6_14,
  px_6_15,
  px_6_16,
  px_6_17,
  px_7_4,
  px_7_5,
  px_7_6,
  px_7_7,
  px_7_8,
  px_7_9,
  px_7_10,
  px_7_11,
  px_7_12,
  px_7_13,
  px_7_14,
  px_7_15,
  px_7_16,
  px_7_17,
  px_8_13,
  px_8_14,
  px_8_15,
  px_8_16,
  px_8_17,
  px_9_10,
  px_9_11,
  px_9_12,
  px_9_13,
  px_9_14,
  px_9_15,
  px_9_16,
  px_9_17,
  px_10_7